In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle as pkl
with open("/home/opc/eipy/tadpole/tadpole_data_time_imptn_norm_v1.pickle", "rb") as file:
    data = pkl.load(file)
with open("/home/opc/eipy/tadpole/tadpole_labels_time_imptn_norm_v1.pickle", "rb") as file:
    labels = pkl.load(file)

In [3]:
data = [data[k] for k in data.keys()]

In [4]:
for k,v in labels.items():
    labels[k] = v.reset_index(drop=True)

In [5]:
#intermediate transformation to make sure labels are ordered correctly in time
labels = pd.DataFrame(labels)

labels = labels.to_numpy()
labels

array([['NL', 'NL', 'NL', 'NL', 'NL'],
       ['MCI', 'MCI', 'MCI', 'NL', 'MCI'],
       ['NL', 'NL', 'NL', 'NL', 'NL'],
       ...,
       ['NL', 'NL', 'MCI', 'MCI', 'MCI'],
       ['NL', 'MCI', 'MCI', 'MCI', 'MCI'],
       ['NL', 'MCI', 'MCI', 'MCI', 'MCI']], dtype=object)

In [6]:
encoding_dict = {'NL': 0, 'MCI': 1, 'Dementia': 2}
labels = np.vectorize(lambda x: encoding_dict[x])(labels)
labels

array([[0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 1, 1, 1],
       [0, 1, 1, 1, 1],
       [0, 1, 1, 1, 1]])

In [7]:
data_arrays = []
for i, appt_data in enumerate(data):
    combined_appt_data = np.concatenate([modality_data for modality_data in appt_data.values()], axis=1)
    data_arrays.append(combined_appt_data)
    


In [8]:
data_array = np.stack([array for array in data_arrays], axis=1)

In [9]:
dem_freq = np.sum(labels == 2, axis=1)

In [10]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(
            n_splits=5, shuffle=True, random_state=38
        )
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []
for train_index, test_index in cv.split(data_array, dem_freq):
    X_train_list.append(data_array[train_index])
    y_train_list.append(labels[train_index])
    X_test_list.append(data_array[test_index])
    y_test_list.append(labels[test_index])
    

/home/opc/.venv/lib64/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [11]:
X_train_list = [X_train[:,:-1,:] for X_train in X_train_list]
X_test_list = [X_test[:,:-1,:] for X_test in X_test_list]
y_train_list = [y_train[:, 1:] for y_train in y_train_list]
y_test_list = [y_test[:, 1:] for y_test in y_test_list]

In [12]:
y_train_categorical = [np.eye(3)[y_train] for y_train in y_train_list]

In [29]:
from keras.models import Sequential
from keras.layers import LSTM,Dense
import tensorflow as tf
y_preds = []
for i in range(5):
    tf.random.set_seed(38)
    lstm = Sequential()
    lstm.add(LSTM(units=120, input_shape=(4,331), return_sequences=True)) #units=80+10*(i+1)
    lstm.add(Dense(units=3, activation='softmax'))
    lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #loss='categorical_crossentropy'

    lstm.fit(X_train_list[i], y_train_categorical[i], epochs=100)
    y_preds.append(lstm.predict(X_test_list[i]))

Epoch 1/100
18/18 [==============================] - 1s 4ms/step - loss: 0.9848 - accuracy: 0.5838
Epoch 2/100
18/18 [==============================] - 0s 4ms/step - loss: 0.9171 - accuracy: 0.5887
Epoch 3/100
18/18 [==============================] - 0s 4ms/step - loss: 0.8823 - accuracy: 0.6016
Epoch 4/100
18/18 [==============================] - 0s 4ms/step - loss: 0.8781 - accuracy: 0.5922
Epoch 5/100
18/18 [==============================] - 0s 4ms/step - loss: 0.8714 - accuracy: 0.5931
Epoch 6/100
18/18 [==============================] - 0s 4ms/step - loss: 0.8565 - accuracy: 0.6056
Epoch 7/100
18/18 [==============================] - 0s 4ms/step - loss: 0.8441 - accuracy: 0.6181
Epoch 8/100
18/18 [==============================] - 0s 4ms/step - loss: 0.8498 - accuracy: 0.6074
Epoch 9/100
18/18 [==============================] - 0s 4ms/step - loss: 0.8394 - accuracy: 0.6163
Epoch 10/100
18/18 [==============================] - 0s 4ms/step - loss: 0.8281 - accuracy: 0.6257
Epoch 11/

In [30]:
y_preds_argmax = [np.argmax(pred, axis=-1) for pred in y_preds]

In [31]:
from sklearn.metrics import classification_report
final_f1_dem = []
for i in range(len(y_preds_argmax)):
    print(f" \n FOLD {i+1} \n")
    for j in range(4):
        print(classification_report(y_pred=y_preds_argmax[i][:,j], y_true=y_test_list[i][:,j]))

 
 FOLD 1 

              precision    recall  f1-score   support

           0       0.36      0.21      0.27        38
           1       0.70      0.86      0.77        97
           2       0.00      0.00      0.00         6

    accuracy                           0.65       141
   macro avg       0.36      0.36      0.35       141
weighted avg       0.58      0.65      0.60       141

              precision    recall  f1-score   support

           0       0.32      0.26      0.29        34
           1       0.70      0.79      0.74        96
           2       0.00      0.00      0.00        11

    accuracy                           0.60       141
   macro avg       0.34      0.35      0.34       141
weighted avg       0.55      0.60      0.57       141

              precision    recall  f1-score   support

           0       0.35      0.25      0.29        36
           1       0.56      0.57      0.57        79
           2       0.17      0.23      0.20        26

    accu

/home/opc/.venv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/opc/.venv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
from sklearn.metrics import precision_recall_fscore_support
dem_f1 = []
for i in range(len(y_preds_argmax)):
    precision, recall, f1, support = precision_recall_fscore_support(y_pred=y_preds_argmax[i][:,-1], y_true=y_test_list[i][:,-1])
    dem_f1.append(f1[2])

np.mean(dem_f1), np.std(dem_f1)

(0.293448893080697, 0.050542544169332654)